In [1]:
import tqdm
import pickle
import pandas as pd
from neo4j import GraphDatabase

In [3]:
df_pw = pd.read_csv("pw.csv", delimiter=",")

In [4]:
db = GraphDatabase.driver("neo4j://127.0.0.1:7687", auth=(df_pw["name"][0], df_pw["password"][0]))

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import pymysql

db_url = {
    'database': 'DarknetScrape',
    'drivername': 'mysql',
    'username': df_pw["name"][0],
    'password': df_pw["password"][0],
    'host': '127.0.0.1',
    'port': 3306,
    'query': {'charset': 'utf8'},
}

db_connection = create_engine(URL(**db_url), encoding="utf8")

c:\users\d067608\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:15: SADeprecationWarning: Calling URL() directly is deprecated and will be disabled in a future release.  The public constructor for URL is now the URL.create() method.
  from ipykernel import kernelapp as app


In [6]:
query = """SELECT
	reviews.price_eur as price_eur_review,
	items.price_eur as price_eur_items,
	items.price_btc ,
	reviews.`time`,
	vendors.vendor_id ,
	reviews.item_id
FROM
	DarknetScrape.item_reviews as reviews
JOIN DarknetScrape.items as items on
	reviews.item_id = items.item_id
join DarknetScrape.vendors vendors on
	vendors.vendor_id = items.vendor_id
join DarknetScrape.market market on
	market.market_name = vendors.market
WHERE
	reviews.positive = 1
	and market.market_name = "Berlusconi Market"
"""

df = pd.read_sql(query, con=db_connection)
df.head()

,price_eur_review,price_eur_items,price_btc,time,vendor_id,item_id
0,208.82,130.0,0.015269,2019-07-07 18:40:42,b5f6b728cec773313da84887919cef33,04906d05a940
1,160.24,130.0,0.015269,2019-07-06 22:10:17,b5f6b728cec773313da84887919cef33,04906d05a940
2,125.91,130.0,0.015269,2019-06-12 17:41:47,b5f6b728cec773313da84887919cef33,04906d05a940
3,119.83,130.0,0.015269,2019-06-08 10:03:15,b5f6b728cec773313da84887919cef33,04906d05a940
4,117.43,130.0,0.015269,2019-06-06 08:36:34,b5f6b728cec773313da84887919cef33,04906d05a940


In [7]:
sess = db.session()
result = sess.run("MATCH (n:Transaction) Where date ({year: 2019, month: 7}) >n.date >= date({year: 2017, month: 8}) RETURN n LIMIT 25")
results = []
for r in result:
    results.append(r[0])
df_neo4j = pd.DataFrame(results)